In [1]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient



# Import CRUD Python module
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password) 


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


# get company logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Animal Shelter - Juan Topete'))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Hr(),
    html.Div(className='row',
             style={"width": "50%"},
             children=[
                 # Water Rescue
                 # Mountain or Wilderness Rescue
                 # Disaster Rescue or Individual Tracking
                 # Reset (returns all widgets to their original, unfiltered state)
                 dcc.Dropdown(
                     id='filter-type',
                     
                     options=[
                         {"label": "Water Rescue", "value": "Water Rescue"},
                         {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                         {"label": "Disaster Rescue or Individual Tracking", "value": "Disaster Rescue or Individual Tracking"},
                         {"label": "Reset", "value": "Reset"},
                     ],
                     value="Reset"
                 ),

             ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        # format the table so that the columns adjust automatically absed on the cell values
        style_header={
            'whiteSpace': 'normal',
            'height': 'auto',
        },
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,                  # allow editing if true
        filter_action="native",          # allow native column filtering
        sort_action="native",            # allow column sorting
        sort_mode="multi",               # allow multiple columns to be sorted
        column_selectable=False,         # allow multiple columns to be selected
        row_selectable="single",            # allow rows being selected
        row_deletable=False,             # disallow rows being deleted
        selected_columns=[],
        selected_rows=[],
        page_action="native",            # enable pagination
        page_current=0,                  # start at the first page on load
        page_size=10,                    # set the number of rows per page to 10

    ),
    html.Br(),
    html.Hr(),
    # create a div for the map data
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.Div([
                 # Dropdown menu for pie chart
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'},       
                            {'label': 'Outcome Type', 'value': 'outcome_type'}
                        ],
                        value='breed',
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"10%"}
                ),
                # Pie chart
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    
                ),
        html.Div(
                    id='map-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    )
                
     ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([
    Output('datatable-id','data'),
    Output('datatable-id','columns'),
    Output('datatable-id','selected_rows')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Main Dashboard Options
    # 1. Water Rescue
    # 2. Mountain or Wilderness Rescue
    # 3. Disaster Rescue or Individual Tracking
    # 4. Reset Widgets
    
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    elif filter_type == "Water Rescue":
        
        df=pd.DataFrame.from_records(shelter.read({
                        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                        "sex_upon_outcome": "Intact Female",
                        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
            ))
    
    elif filter_type == "Mountain or Wilderness Rescue":
        # German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler
        # Intact Male
        # 26 weeks to 156 weeks
        # {"breed": {$in: ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Maale", "age_upon_outcome_in_weeks": {$gte: 26, $lte: 156}}
        df=pd.DataFrame.from_records(shelter.read({
                        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                        "sex_upon_outcome": "Intact Male",
                        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
                    
        ))
    
    elif filter_type == "Disaster Rescue or Individual Tracking":
        # Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler
        # Intact Male
        # 20 weeks to 300 weeks
        # {"breed": {$in: ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {$gte: 20, $let: 300}}
        df=pd.DataFrame.from_records(shelter.read({
                        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                        "sex_upon_outcome": "Intact Male",
                        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}},
        ))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    
    data = df.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    selected_rows = [0]

    return (data, columns, selected_rows)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#d2ffd7' 
    } for i in selected_columns]

# Generate pie chart from dropdown menu selection
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('pie_dropdown', 'value')])
def update_graphs(viewData, value):
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(data_frame = dffPie, names = value)
        )    
    ]


@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows):
    #austin Texas is [30.75, -97.48]
    
    #create the views
    if not virtualRows: #build a default view if there are no selected lines
        markerArray = (30.75,-97.48) #default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: #build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows]) #convert the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) #strip out the lat
        coordLong = float(dff['location_long'].to_string().split()[1]) #strip out the long
        markerArray = (coordLat, coordLong) #build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    #return the map with a child marker
    #marker is set to the values found in markerArray
    #map centers/moves to view the new marker instead of holding a fixed center
    return [dl.Map(style={'width': '700px', 'height': '450px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:18452/


/home/juantopete_snhu/.local/lib/python3.9/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

